## Testing out Entropy between Distribution

We extract the attention distributions from our pre-trained BERT model and observe the difference in entropy between distributions extracted from sentences with epistemological and framing bias

In [25]:
import sys; sys.path.append("../../../..") #NOTE: changing from the basic path we use for other experiments
import torch 
from src.experiment import AttentionExperiment
from src.dataset import ExperimentDataset
from src.params import Params

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
params = Params.read_params("params.json")
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))

layers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [28]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

04/03/2020 18:07:04 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 4812.40it/s]


### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [29]:
attention_dataloader = dataset.return_dataloader(batch_size=params.intermediary_task['attention']['attention_extraction_batch_size']) 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True)

04/03/2020 18:07:04 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/03/2020 18:07:05 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/03/2020 18:07:05 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpjtow6906
04/03/2020 18:07:08 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

Instantiated joint model with pretrained weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [30]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [31]:
dataset

Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label'])

In [32]:
bias_indices = []
for entry in dataset: 
    idx = entry['pre_tok_label_ids'].to(dtype=torch.int).flatten().tolist().index(1)
    bias_indices.append(idx)

In [33]:
from src.utils.attention_utils import return_idx_attention_dist
attention_dist = return_idx_attention_dist(attention_scores, bias_indices)

In [34]:
attention_dist_dict = {}
for attention_dict in attention_dist: 
    for key, val in attention_dict.items():
        if key not in attention_dist_dict: 
            attention_dist_dict[key] = val
        # otherwise we need to concatenate the distributions together
        else: 
            prev_val = attention_dist_dict[key]
            attention_dist_dict[key] = torch.cat((prev_val, val), dim=0)

In [35]:
labels = dataset.get_val('bias_label')

In [36]:
labels_0_indices = (labels == 0).nonzero()
labels_1_indices = labels.nonzero()

In [39]:
from scipy.stats import entropy
from src.utils.attention_utils import window_attention_dist

In [64]:
avg_entropies_0 = []
avg_entropies_1 = []
for key, attention_dist in attention_dist_dict.items():
    attention_dist_0 = attention_dist[labels_0_indices].squeeze() # epistemological 
    attention_dist_1 = attention_dist[labels_1_indices].squeeze() # framing
    entropy_0 = entropy(attention_dist_0.t())
    entropy_1 = entropy(attention_dist_1.t())
    avg_entropy_0 = sum(entropy_0)/len(entropy_0)
    avg_entropy_1 = sum(entropy_1)/len(entropy_1)
    print("Layer {}: Entropy Epistemological: {} \t Entropy Framing {}".format(key, avg_entropy_0, avg_entropy_1))
    avg_entropies_0.append(avg_entropy_0)
    avg_entropies_1.append(avg_entropy_1)

print("\n")
print("Average Entropy Over All Epistemological: {}".format(sum(avg_entropies_0)/len(avg_entropies_0)))
print("Average Entropy Over All Framing: {}".format(sum(avg_entropies_1)/len(avg_entropies_1)))

Layer 0: Entropy Epistemological: 2.9699798211578496 	 Entropy Framing 3.010976932906165
Layer 1: Entropy Epistemological: 2.6974301091895616 	 Entropy Framing 2.7149727156596817
Layer 2: Entropy Epistemological: 2.5900012077378833 	 Entropy Framing 2.6097387750747756
Layer 3: Entropy Epistemological: 2.6268901026938574 	 Entropy Framing 2.633833535492714
Layer 4: Entropy Epistemological: 2.8849388735353454 	 Entropy Framing 2.9235627281254737
Layer 5: Entropy Epistemological: 2.6848199732047466 	 Entropy Framing 2.7148549627200724
Layer 6: Entropy Epistemological: 2.6063626413502967 	 Entropy Framing 2.6157466684068953
Layer 7: Entropy Epistemological: 2.482777354145838 	 Entropy Framing 2.5211734624919044
Layer 8: Entropy Epistemological: 2.778633512741278 	 Entropy Framing 2.791751378862729
Layer 9: Entropy Epistemological: 2.872147674402915 	 Entropy Framing 2.9267627881665534
Layer 10: Entropy Epistemological: 2.4515067713319763 	 Entropy Framing 2.3189149431407157
Layer 11: Entro

In [85]:
avg_entropies_0 = []
avg_entropies_1 = []
for key, attention_dist in attention_dist_dict.items():
    windowed_attention_dist = window_attention_dist(attention_dist, bias_indices, window_size=5)
    normalized_windowed_attention_dist = windowed_attention_dist / torch.sum(windowed_attention_dist, dim=1, keepdim=True)
    
    attention_dist_0 = normalized_windowed_attention_dist[labels_0_indices].squeeze() # epistemological 
    attention_dist_1 = normalized_windowed_attention_dist[labels_1_indices].squeeze() # framing
    entropy_0 = entropy(attention_dist_0.t())
    entropy_1 = entropy(attention_dist_1.t())
    avg_entropy_0 = sum(entropy_0)/len(entropy_0)
    avg_entropy_1 = sum(entropy_1)/len(entropy_1)
    print("Layer {}: Entropy Epistemological: {} \t Entropy Framing {}".format(key, avg_entropy_0, avg_entropy_1))
    avg_entropies_0.append(avg_entropy_0)
    avg_entropies_1.append(avg_entropy_1)

print("\n")
print("Average Entropy Over All Epistemological: {}".format(sum(avg_entropies_0)/len(avg_entropies_0)))
print("Average Entropy Over All Framing: {}".format(sum(avg_entropies_1)/len(avg_entropies_1)))

Layer 0: Entropy Epistemological: 2.104026387545688 	 Entropy Framing 2.127955666903792
Layer 1: Entropy Epistemological: 1.990709040775772 	 Entropy Framing 1.9842406053261217
Layer 2: Entropy Epistemological: 1.9090724055432091 	 Entropy Framing 1.8980930456386997
Layer 3: Entropy Epistemological: 1.9572476823467853 	 Entropy Framing 1.9513696520199328
Layer 4: Entropy Epistemological: 2.029124992938081 	 Entropy Framing 2.050512483260902
Layer 5: Entropy Epistemological: 1.8802496347545592 	 Entropy Framing 1.9070896785247502
Layer 6: Entropy Epistemological: 1.8754660698993146 	 Entropy Framing 1.8614262371814896
Layer 7: Entropy Epistemological: 1.7494005757915088 	 Entropy Framing 1.7689679226851815
Layer 8: Entropy Epistemological: 1.9188331946853763 	 Entropy Framing 1.8532549339562214
Layer 9: Entropy Epistemological: 1.9267516727289877 	 Entropy Framing 1.9557632363488522
Layer 10: Entropy Epistemological: 1.5355080543470776 	 Entropy Framing 1.29558250528251
Layer 11: Entrop